# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [1]:
# Import libraries
import numpy as np
import pandas as pd

In [2]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print( "Student data read successfully!" )
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [3]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = student_data.shape[0]
n_features = student_data.shape[1] - 1
n_passed = student_data[student_data['passed'] == 'yes'].shape[0]
n_failed = student_data[student_data['passed'] == 'no'].shape[0]
grad_rate = float( n_passed ) / n_students*100
print( "Total number of students: {}".format( n_students ) )
print( "Number of students who passed: {}".format( n_passed ) )
print( "Number of students who failed: {}".format( n_failed ) )
print( "Number of features: {}".format( n_features ) )
print( "Graduation rate of the class: {:.2f}%".format( grad_rate ) )

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 67.09%


### Reviwer Pro Tip:
When dealing with the new data set it is good practice to assess its specific characteristics and implement the cross validation technique tailored on those very characteristics, in our case there are two main elements:

    1. Our dataset is small.
    2. Our dataset is slightly unbalanced. (There are more passing students than on passing students)

We could take advantage of K-fold cross validation to exploit small data sets. Even though in this case it might not be necessary, should we have to deal with heavily unbalance datasets, we could address the unbalanced nature of our data set using Stratified K-Fold and Stratified Shuffle Split Cross validation, as stratification is preserving the preserving the percentage of samples for each class.

http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.StratifiedShuffleSplit.html

http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.StratifiedKFold.html

## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [4]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print( "Feature column(s):-\n{}".format(feature_cols) )
print( "Target column: {}".format(target_col) )

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print( "\nFeature values:-" )
print( X_all.head() )  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [5]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print( "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns)) )

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [6]:
def shuffle_split(train_size=300):
    # First, decide how many training vs test samples you want
    num_all = student_data.shape[0]  # same as len(student_data)
    num_train = train_size  # 300 is about 75% of the data
    num_test = num_all - num_train

    # TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
    # Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
    y = student_data["passed"]

    from sklearn.cross_validation import train_test_split
    X_train, X_test, y_train, y_test = train_test_split( X_all, y, train_size=num_train, random_state=np.random.randint(1000) )
    
    return X_train, y_train, X_test, y_test 

X_train, y_train, X_test, y_test = shuffle_split()

print( "Training set: {} samples".format(X_train.shape[0]) )
print( "Test set: {} samples".format(X_test.shape[0]) )
# Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

In [7]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    print( "Training {}...".format(clf.__class__.__name__) )
    start = time.time()
    clf.fit(X_train, np.ravel(y_train)) # note: Had to use np. ravel to convert 1D column vector to 1D array.
    end = time.time()
    print( "Done!\nTraining time (secs): {:.3f}".format(end - start) )

# TODO: Choose a model, import it and instantiate an object
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()

# Fit model to training data
train_classifier( clf, X_train, y_train )  # note: using entire training set here
print( clf )  # you can inspect the learned model by printing it

Training LogisticRegression...
Done!
Training time (secs): 0.005
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


In [8]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print( "Predicting labels using {}...".format(clf.__class__.__name__) )
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print( "Done!\nPrediction time (secs): {:.3f}".format(end - start) )
    return f1_score(target.values, y_pred, pos_label='yes')

train_f1_score = predict_labels(clf, X_train, y_train)
print( "F1 score for training set: {}".format(train_f1_score) )

Predicting labels using LogisticRegression...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.8411633109619687


In [9]:
# Predict on test data
print( "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test)) )

Predicting labels using LogisticRegression...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.7938931297709924


In [10]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print( "------------------------------------------" )
    print( "Training set size: {}".format(len(X_train)) )
    train_classifier(clf, X_train, y_train)
    print( "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train)) )
    print( "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test)) )

# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant
train_predict(clf, X_train, y_train, X_test, y_test)

------------------------------------------
Training set size: 300
Training LogisticRegression...
Done!
Training time (secs): 0.005
Predicting labels using LogisticRegression...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.8411633109619687
Predicting labels using LogisticRegression...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.7938931297709924


In [29]:
# TODO: Train and predict using two other models

def multi_data( train_size=300, data_size=5 ):
    data = [[0]*4]*data_size
    
    for ds in range( data_size ):
        data[ds][0], data[ds][1], data[ds][2],  data[ds][3] = shuffle_split(train_size)
    
    return data

def multi_classify( classifiers, data, train_sizes ):
    for train_size in train_sizes:        
        for clf in classifiers:
            sum_train_time = 0.0
            sum_pred_time = 0.0
            sum_f1_train = 0.0
            sum_f1_test = 0.0

            for d in range( len( data ) ):     
                X_train = data[d][0]
                y_train = data[d][1]
                X_test = data[d][2]
                y_test = data[d][3]

                train_time_start = time.time()
                clf.fit(X_train[0:train_size], np.ravel( y_train[0:train_size] ) )
                train_time_end = time.time()

                pred_train_y = clf.predict( X_train[0:train_size] )

                pred_time_start = time.time()
                pred_test_y = clf.predict( X_test )
                pred_time_end = time.time()

                sum_train_time += train_time_end - train_time_start
                sum_pred_time += pred_time_end - pred_time_start
                sum_f1_train += f1_score( y_train[0:train_size].values, pred_train_y, pos_label='yes' )
                sum_f1_test += f1_score( y_test.values, pred_test_y, pos_label='yes' )

            avg_train_time = sum_train_time / len( data )
            avg_pred_time = sum_pred_time / len( data )
            avg_f1_train = sum_f1_train / len( data )
            avg_f1_test = sum_f1_test / len( data )        

            print( "------------------------------------------" )
            print( "Model: {}...".format( clf.__class__.__name__ ) )
            print( "Training set size: {}".format( train_size ) )
            print( "Avg training time (secs): {:.4f}".format( avg_train_time ) )
            print( "Avg prediction time (secs): {:.4f}".format( avg_pred_time ) )
            print( "Avg F1 score for training set: {:.4f}".format( avg_f1_train ) )
            print( "Avg F1 score for test set: {:.4f}".format( avg_f1_test ) )

        
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB

sizes = [100, 200, 300]

classifiers = [
    #LogisticRegression(),
    #KNeighborsClassifier(10),
    SVC(kernel="rbf", C=0.025),
    #DecisionTreeClassifier(max_depth=10),
    #RandomForestClassifier(n_estimators=10),
    AdaBoostClassifier(n_estimators=5),
    MultinomialNB(alpha=10),
]

for clf in classifiers:
    print( clf )
    print( '\n' )

#data
data_size = 10
data = multi_data( 300, data_size )

#classification
multi_classify( classifiers, data, sizes )

SVC(C=0.025, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=5, random_state=None)


MultinomialNB(alpha=10, class_prior=None, fit_prior=True)


------------------------------------------
Model: SVC...
Training set size: 100
Avg training time (secs): 0.0020
Avg prediction time (secs): 0.0011
Avg F1 score for training set: 0.7654
Avg F1 score for test set: 0.8344
------------------------------------------
Model: AdaBoostClassifier...
Training set size: 100
Avg training time (secs): 0.0080
Avg prediction time (secs): 0.0009
Avg F1 score for training set: 0.8369
Avg F1 score for test set: 0.8531
------------------------------------------
Model: MultinomialNB...
Training set size: 100
Avg training time (secs): 0.0017
Avg pre

## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

In [32]:
# TODO: Fine-tune your model and report the best F1 score
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
f1_scorer = make_scorer(f1_score, pos_label="yes")

parameters = {'alpha':[0.01, 0.1, 1, 5, 10, 25, 50, 75, 100, 125, 150, 200]}

for d in range ( len( data ) ):
    X_train = data[d][0]
    y_train = data[d][1]
    #X_test = data_300[d][2]
    #y_test = data_300[d][3]
    
    grid_search = GridSearchCV( classifiers[2], parameters, scoring=f1_scorer )
    grid_search.fit( X_train, np.ravel( y_train ) )
    
    clf_best = grid_search.best_estimator_
    print( clf_best )

multi_classify([clf_best], data, sizes)

MultinomialNB(alpha=25, class_prior=None, fit_prior=True)
MultinomialNB(alpha=25, class_prior=None, fit_prior=True)
MultinomialNB(alpha=25, class_prior=None, fit_prior=True)
MultinomialNB(alpha=25, class_prior=None, fit_prior=True)
MultinomialNB(alpha=25, class_prior=None, fit_prior=True)
MultinomialNB(alpha=25, class_prior=None, fit_prior=True)
MultinomialNB(alpha=25, class_prior=None, fit_prior=True)
MultinomialNB(alpha=25, class_prior=None, fit_prior=True)
MultinomialNB(alpha=25, class_prior=None, fit_prior=True)
MultinomialNB(alpha=25, class_prior=None, fit_prior=True)
------------------------------------------
Model: MultinomialNB...
Training set size: 100
Avg training time (secs): 0.0016
Avg prediction time (secs): 0.0002
Avg F1 score for training set: 0.7654
Avg F1 score for test set: 0.8344
------------------------------------------
Model: MultinomialNB...
Training set size: 200
Avg training time (secs): 0.0017
Avg prediction time (secs): 0.0002
Avg F1 score for training set: 0

### Pro Tip (Advanced) 
You could actually go well beyond grid search and implement ‘pipelines’ where the whole machine learning process becomes 'grid-searchable' and you can parameterize and search the whole process though cross validation.

http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html

And yes you can try out several algorithms automatically as well too! Watch out though this is pretty advanced stuff, here is a great, informative, top notch tutorial from Zac Sewart!

http://zacstewart.com/2014/08/05/pipelines-of-featureunions-of-pipelines.html
